In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 23 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

29 March 2023 ; 12:25:32


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,32,0,9.4,1.5,3.4,0.440,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,1,1
1,Bobby Portis,PF,27,MIL,64,20,26.0,5.6,11.3,0.496,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,0,1
4,Goran Dragić,PG,36,MIL,52,0,15.3,2.4,5.8,0.422,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,0,1
6,Jae Crowder,SF,32,MIL,12,0,18.2,1.9,4.0,0.479,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,0,1
7,Jevon Carter,PG,27,MIL,75,36,22.2,3.0,7.0,0.427,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,25.7,63,0,BOS,18.1,2.473016,7.028571,14.6,2,2.2,4.425397,0.310415
1,Immanuel Quickley,28.5,75,17,NYK,15.8,2.077333,6.560000,14.1,5,1.1,4.264000,0.152060
2,Bobby Portis,26.0,64,20,MIL,17.7,1.409375,6.278125,13.8,1,1.0,2.818750,0.137948
3,Austin Reaves,28.3,57,15,LAL,14.1,1.294737,6.185965,12.3,9,0.1,4.315789,0.065657
4,Christian Wood,26.3,63,17,DAL,20.7,1.952381,6.117460,16.9,10,1.9,3.384127,0.055122


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)